## Descrição

Baixando e formatadando dados para o Tech Challenge do site:
    
- http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01

## Importando bibliotecas

In [1]:
import os
import pandas as pd

## Funções usadas

In [45]:
def formata_df(df: pd.DataFrame, tipo: str) -> pd.DataFrame:
    
    """Função usada para formatar o dado
    """
    
    #PEGA COLUNAS QUANTIDADE
    colunas_qtd = [i for i in df.columns if '.' not in i]
    colunas_qtd.remove('Id')
    
    #PEGA COLUNAS VALOR
    colunas_valor = [i for i in df.columns if '.' in i]
    colunas_valor.insert(0, 'País')
    
    #DF QUANTIDADE
    df_qtd = df[colunas_qtd]
    df_qtd_melted = df_qtd.melt(id_vars=['País'], value_vars=df_qtd.columns)
    df_qtd_melted.columns = ['pais_destino', 'ano', 'quantidade']
    
    #DF VALOR
    df_valor = df[colunas_valor]
    df_valor.columns = colunas_qtd #RENOMEANDO COLUNAS PARA FACILITAR O MATCH DEPOIS

    df_valor_melted = df_valor.melt(id_vars=['País'], value_vars=df_valor.columns)
    df_valor_melted.columns = ['pais_destino', 'ano', 'valor']
    
    #JUNTANDO AS INFORMAÇÕES
    df_merged = pd.merge(df_qtd_melted,
                         df_valor_melted,
                         left_on=['pais_destino', 'ano'],
                         right_on=['pais_destino', 'ano'])

    #CRIANDO COLUNAS
    df_merged['pais_origem'] = 'Brasil'
    df_merged['tipo'] = tipo
    
    return df_merged[['ano', 'pais_origem', 'pais_destino', 'tipo', 'quantidade', 'valor']]

***

Usado para criar colunas com os nomes corretos (anos e tipo do vinho)

In [2]:
nome_colunas = [i for i in range(1970, 2022)]
nome_colunas.insert(0, 'VINHO')

## Formatação

### 1. Produção

In [3]:
producao_linhas_para_remover = ['VINHO DE MESA', 'VINHO FINO DE MESA (VINÍFERA)', 'SUCO', 'DERIVADOS']

In [4]:
df_producao = pd.read_csv('../dados/raw/producao/Producao.csv', delimiter=';',
                         header=None)

df_producao = df_producao[df_producao.columns[2:]]
df_producao.columns = nome_colunas
df_producao = df_producao.query('VINHO not in @producao_linhas_para_remover').reset_index()

df_producao.loc[df_producao.index[0:3], 'TIPO'] = 'VINHO DE MESA'
df_producao.loc[df_producao.index[3:6], 'TIPO'] = 'VINHO FINO DE MESA (VINÍFERA)'
df_producao.loc[df_producao.index[6:11], 'TIPO'] = 'SUCO'
df_producao.loc[df_producao.index[11:], 'TIPO'] = 'DERIVADOS'
df_producao.head()

,index,VINHO,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,TIPO
0,1,Tinto,174224052,121133369,118180926,88589019,146544484,144274134,118360170,154801826,...,163111797,157776363,169811472,75279191,1365957,188270142,121045115,103916391,146075996,VINHO DE MESA
1,2,Branco,748400,1160500,1812367,243900,4138768,1441507,1871473,4954387,...,32066403,37438069,39557250,10727099,217527985,29229970,22032828,19568734,26432799,VINHO DE MESA
2,3,Rosado,42236152,31970782,26960004,27877426,43192093,31685568,24333795,35603565,...,1726022,958691,939838,312725,36121245,875524,1551794,715289,1391200,VINHO DE MESA
3,5,Tinto,7591557,7265666,6782837,3419625,10047658,14731106,12405154,16131758,...,23156458,17208996,16745896,8774847,21442212,19118254,17389377,15451883,20433249,VINHO FINO DE MESA (VINÍFERA)
4,6,Branco,15562889,15655709,13289304,7930070,18927471,21933695,19874659,20409231,...,21906349,20054804,19561966,8705066,21928400,18297257,18193055,15487915,20867999,VINHO FINO DE MESA (VINÍFERA)


##### Escrevendo versão "formatada"

In [5]:
df_producao.to_csv('../dados/formatted/df_producao.csv', encoding='utf-8', sep='|', index=False)

***

### 2. Processamento

In [6]:
processamento_linhas_para_remover = ['TINTAS', 'BRANCAS E ROSADAS', 'BRANCAS']

#### Viniferas

In [7]:
df_processamento_viniferas = pd.read_csv('../dados/raw/processamento/ProcessaViniferas.csv',
                              delimiter='\t')

df_processamento_viniferas = df_processamento_viniferas[df_processamento_viniferas.columns[2:]]
df_processamento_viniferas = df_processamento_viniferas.query('cultivar not in @processamento_linhas_para_remover').reset_index()

df_processamento_viniferas.loc[df_processamento_viniferas.index[0:68], 'TIPO'] = 'TINTAS'
df_processamento_viniferas.loc[df_processamento_viniferas.index[68:], 'TIPO'] = 'BRANCAS E ROSADAS'

df_processamento_viniferas = df_processamento_viniferas.rename(columns={'cultivar': 'VINHO'})

df_processamento_viniferas.head()

,index,VINHO,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,TIPO
0,1,Alicante Bouschet,0,0,0,0,0,0,0,0,...,1524728,1456305,1519576,908841,2040198,2103844,nd,2272985,811140,TINTAS
1,2,Ancelota,0,0,0,0,0,0,0,0,...,1137943,937844,773526,179028,733907,492106,nd,481402,6513974,TINTAS
2,3,Aramon,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,TINTAS
3,4,Alfrocheiro,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,TINTAS
4,5,Arinarnoa,0,0,0,0,0,0,0,0,...,65001,56666,61817,17691,69757,44775,nd,90860,2785609,TINTAS


#### Americanas e hibridas

In [8]:
df_processamento_americanas = pd.read_csv('../dados/raw/processamento/ProcessaAmericanas.csv',
                              delimiter=';')

df_processamento_americanas = df_processamento_americanas[df_processamento_americanas.columns[2:]]
df_processamento_americanas = df_processamento_americanas.query('cultivar not in @processamento_linhas_para_remover').reset_index()

df_processamento_americanas.loc[df_processamento_americanas.index[0:34], 'TIPO'] = 'TINTAS'
df_processamento_americanas.loc[df_processamento_americanas.index[34:], 'TIPO'] = 'BRANCAS E ROSADAS'

df_processamento_americanas = df_processamento_americanas.rename(columns={'cultivar': 'VINHO'})

df_processamento_americanas.head()

,index,VINHO,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,TIPO
0,1,Bacarina,82899,106962,67464,58690,138158,101454,57297,0,...,2990,3900,0,0,0,0,nd,0,0,TINTAS
1,2,Bailey,0,0,0,0,0,0,0,0,...,756000,991449,963159,442784,1370092,539742,nd,534981,4092669,TINTAS
2,3,Bordo,7242197,7227090,6530686,5584243,13341412,16023998,12725233,18714617,...,102788361,113008320,137467196,60976531,160146475,158405972,nd,129978861,117655879,TINTAS
3,4,Bourdin (S),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,TINTAS
4,5,BRS Carmen,0,0,0,0,0,0,0,0,...,605728,1553111,2891792,2964574,6771269,7472984,nd,11540681,13771662,TINTAS


#### Uvas de mesa

In [9]:
df_processamento_uvas_de_mesa = pd.read_csv('../dados/raw/processamento/ProcessaMesa.csv',
                              delimiter=';')

df_processamento_uvas_de_mesa = df_processamento_uvas_de_mesa[df_processamento_uvas_de_mesa.columns[2:]]
df_processamento_uvas_de_mesa = df_processamento_uvas_de_mesa.query('cultivar not in @processamento_linhas_para_remover').reset_index()

df_processamento_uvas_de_mesa.loc[df_processamento_uvas_de_mesa.index[0:2], 'TIPO'] = 'TINTAS'
df_processamento_uvas_de_mesa.loc[df_processamento_uvas_de_mesa.index[2:], 'TIPO'] = 'BRANCAS'

df_processamento_uvas_de_mesa = df_processamento_uvas_de_mesa.rename(columns={'cultivar': 'VINHO'})

df_processamento_uvas_de_mesa.head()

,index,VINHO,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,TIPO
0,1,Alphonse Lavallee,31878,2333,170,7690,124762,74293,23684,24430,...,0,0,0,0,0,0,nd,0,0,TINTAS
1,2,Moscato de Hamburgo,25098,41057,4258,1249,801,109438,64828,89161,...,75362,65850,108797,51310,85510,62567,nd,63474,21732,TINTAS
2,4,Cardinal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,BRANCAS
3,5,Golden Queen,167,8654,657,7312,8467,6805,0,2391,...,0,0,0,0,0,0,nd,0,0,BRANCAS
4,6,Patrícia,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,BRANCAS


#### Concatenando todos os DFs


In [10]:
df_processamento = pd.concat([df_processamento_viniferas,
                              df_processamento_americanas,
                              df_processamento_uvas_de_mesa])
df_processamento.head()

,index,VINHO,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,TIPO
0,1,Alicante Bouschet,0,0,0,0,0,0,0,0,...,1524728,1456305,1519576,908841,2040198,2103844,nd,2272985,811140,TINTAS
1,2,Ancelota,0,0,0,0,0,0,0,0,...,1137943,937844,773526,179028,733907,492106,nd,481402,6513974,TINTAS
2,3,Aramon,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,TINTAS
3,4,Alfrocheiro,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,nd,0,0,TINTAS
4,5,Arinarnoa,0,0,0,0,0,0,0,0,...,65001,56666,61817,17691,69757,44775,nd,90860,2785609,TINTAS


##### Escrevendo versão "formatada"

In [11]:
df_processamento.to_csv('../dados/formatted/df_processamento.csv', encoding='utf-8', sep='|', index=False)

***

### 3. Comercialização

In [12]:
comercializacao_linhas_para_remover = ['VINHO DE MESA', 'VINHO  FINO DE MESA', 'VINHO ESPECIAL', 'ESPUMANTES ', 'SUCO DE UVAS', 'OUTROS PRODUTOS COMERCIALIZADOS']

In [13]:
df_comercializacao = pd.read_csv('../dados/raw/comercializacao/Comercio.csv',
                                delimiter=';')

df_comercializacao = df_comercializacao[df_comercializacao.columns[2:]]
df_comercializacao.columns = nome_colunas
df_comercializacao = df_comercializacao.query('VINHO not in @comercializacao_linhas_para_remover').reset_index()

df_comercializacao.loc[df_comercializacao.index[0:3], 'TIPO'] = 'VINHO DE MESA'
df_comercializacao.loc[df_comercializacao.index[3:6], 'TIPO'] = 'VINHO FINO DE MESA'

#CASOS ESPECÍFICOS
df_comercializacao.loc[df_comercializacao.index[6], 'TIPO'] = 'VINHO FRIZANTE'
df_comercializacao.loc[df_comercializacao.index[7], 'TIPO'] = 'VINHO ORGÂNICO'
df_comercializacao.loc[df_comercializacao.index[8:11], 'TIPO'] = 'VINHO ESPECIAL'
df_comercializacao.loc[df_comercializacao.index[11:14], 'TIPO'] = 'ESPUMANTES'
df_comercializacao.loc[df_comercializacao.index[14:18], 'TIPO'] = 'SUCO DE UVAS'
df_comercializacao.loc[df_comercializacao.index[18], 'TIPO'] = 'SUCO DE UVAS CONCENTRADO'
df_comercializacao.loc[df_comercializacao.index[19:], 'TIPO'] = 'OUTROS PRODUTOS COMERCIALIZADOS'

df_comercializacao.head()

,index,VINHO,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,TIPO
0,0,Tinto,83300735,98522869,101167932,98196747,77167303,91528090,116407222,116609545,...,188033494,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,VINHO DE MESA
1,1,Rosado,107681,542274,7770851,8425617,8891367,7261777,11748047,15195525,...,1777648,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,VINHO DE MESA
2,2,Branco,14919190,15333888,9438584,9995546,8114654,9241925,11083345,9008044,...,31779668,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,VINHO DE MESA
3,4,Tinto,435354,428927,624499,783508,1616144,2050960,4450570,4504303,...,19121750,15354938,15572632,15228514,12021684,11150517,11433702,18202453,19337862,VINHO FINO DE MESA
4,5,Rosado,183234,227299,403664,1377333,1725564,2130619,3118919,3248565,...,214269,164219,169185,172351,182080,262430,503524,993248,1603537,VINHO FINO DE MESA


##### Escrevendo versão formatada

In [14]:
df_comercializacao.to_csv('../dados/formatted/df_comercializacao.csv', encoding='utf-8', sep='|', index=False)

***

### 4. Importação - PENDENTE
Manter coluna de valor? $

#### Vinhos de mesa

In [54]:
df_importacao_vinhos_de_mesa = pd.read_csv('../dados/raw/importacao/ImpVinhos.csv',
                           delimiter=';')

df_importacao_vinhos_de_mesa = formata_df(df_importacao_vinhos_de_mesa, 'Vinhos de mesa')
df_importacao_vinhos_de_mesa.tail()

,ano,pais_origem,pais_destino,tipo,quantidade,valor
3375,2021,Brasil,Ucrânia,Vinhos de mesa,0,0.0
3376,2021,Brasil,Uruguai,Vinhos de mesa,3788831,10063341.0
3377,2021,Brasil,Não consta na tabela,Vinhos de mesa,0,0.0
3378,2021,Brasil,Não declarados,Vinhos de mesa,0,0.0
3379,2021,Brasil,Outros,Vinhos de mesa,0,0.0


#### Espumantes


In [55]:
df_importacao_espumantes = pd.read_csv('../dados/raw/importacao/ImpEspumantes.csv',
                           delimiter=';')

df_importacao_espumantes = formata_df(df_importacao_espumantes, 'Espumantes')
df_importacao_espumantes.tail()

,ano,pais_origem,pais_destino,tipo,quantidade,valor
2179,2021,Brasil,Ucrânia,Espumantes,0,0
2180,2021,Brasil,Uruguai,Espumantes,8947,47217
2181,2021,Brasil,Não consta na tabela,Espumantes,0,0
2182,2021,Brasil,Não declarados,Espumantes,0,0
2183,2021,Brasil,Outros,Espumantes,0,0


#### Uvas frescas

In [60]:
df_importacao_uvas_frescas = pd.read_csv('../dados/raw/importacao/ImpFrescas.csv',
                           delimiter=';')

df_importacao_uvas_frescas = formata_df(df_importacao_uvas_frescas, 'Uvas frescas')
df_importacao_uvas_frescas[df_importacao_uvas_frescas['ano'] == '2021'].head()

,ano,pais_origem,pais_destino,tipo,quantidade,valor
969,2021,Brasil,Argélia,Uvas frescas,0,0
970,2021,Brasil,Argentina,Uvas frescas,1356735,1696659
971,2021,Brasil,Brasil,Uvas frescas,0,0
972,2021,Brasil,Chile,Uvas frescas,3888723,5404163
973,2021,Brasil,Colômbia,Uvas frescas,0,0


#### Uvas passas

In [62]:
df_importacao_passas = pd.read_csv('../dados/raw/importacao/ImpPassas.csv',
                           delimiter=';')

df_importacao_passas = formata_df(df_importacao_passas, 'Uvas passas')
df_importacao_passas.tail(10)

,ano,pais_origem,pais_destino,tipo,quantidade,valor
1758,2021,Brasil,Peru,Uvas passas,0,0
1759,2021,Brasil,Portugal,Uvas passas,0,0
1760,2021,Brasil,Reino Unido,Uvas passas,0,0
1761,2021,Brasil,Suécia,Uvas passas,0,0
1762,2021,Brasil,Turquia,Uvas passas,186000,311165
1763,2021,Brasil,Uruguai,Uvas passas,0,0
1764,2021,Brasil,Uzbequistão,Uvas passas,0,0
1765,2021,Brasil,Venezuela,Uvas passas,0,0
1766,2021,Brasil,Não consta na tabela,Uvas passas,0,0
1767,2021,Brasil,Outros,Uvas passas,0,0


#### Suco de uva

In [69]:
df_importacao_suco_de_uva = pd.read_csv('../dados/raw/importacao/ImpSuco.csv',
                           delimiter=';')

df_importacao_suco_de_uva = formata_df(df_importacao_suco_de_uva, 'Suco de uva')
df_importacao_suco_de_uva.tail(15)

,ano,pais_origem,pais_destino,tipo,quantidade,valor
1493,2021,Brasil,Israel,Suco de uva,720,969
1494,2021,Brasil,itália,Suco de uva,0,0
1495,2021,Brasil,Japão,Suco de uva,0,0
1496,2021,Brasil,Líbano,Suco de uva,0,0
1497,2021,Brasil,México,Suco de uva,0,0
1498,2021,Brasil,Países Baixos,Suco de uva,0,0
1499,2021,Brasil,Panamá,Suco de uva,0,0
1500,2021,Brasil,Peru,Suco de uva,0,0
1501,2021,Brasil,Portugal,Suco de uva,0,0
1502,2021,Brasil,Reino Unido,Suco de uva,0,0


#### Concatenando todos os DFs

In [70]:
df_importacao = pd.concat([df_importacao_vinhos_de_mesa,
                           df_importacao_espumantes,
                           df_importacao_uvas_frescas,
                           df_importacao_passas,
                           df_importacao_suco_de_uva])

df_importacao.tipo.value_counts()

Vinhos de mesa    3380
Espumantes        2184
Uvas passas       1768
Suco de uva       1508
Uvas frescas       988
Name: tipo, dtype: int64

##### Escrevendo versão "formatada"

In [71]:
df_importacao.to_csv('../dados/formatted/df_importacao_2.csv', encoding='utf-8', sep='|', index=False)

In [75]:
df_importacao[(df_importacao['pais_destino'] == 'Brasil') & (df_importacao['quantidade'] > 0)].head()

,ano,pais_origem,pais_destino,tipo,quantidade,valor
2221,2004,Brasil,Brasil,Vinhos de mesa,2088,16212.0
2286,2005,Brasil,Brasil,Vinhos de mesa,3007,20041.0
2351,2006,Brasil,Brasil,Vinhos de mesa,2930,12527.0
2416,2007,Brasil,Brasil,Vinhos de mesa,3513,19633.0
2481,2008,Brasil,Brasil,Vinhos de mesa,309,1769.0


In [74]:
df_importacao[(df_importacao['pais_destino'] == 'Brasil') & (df_importacao['valor'] > 0)].head()

,ano,pais_origem,pais_destino,tipo,quantidade,valor
2221,2004,Brasil,Brasil,Vinhos de mesa,2088,16212.0
2286,2005,Brasil,Brasil,Vinhos de mesa,3007,20041.0
2351,2006,Brasil,Brasil,Vinhos de mesa,2930,12527.0
2416,2007,Brasil,Brasil,Vinhos de mesa,3513,19633.0
2481,2008,Brasil,Brasil,Vinhos de mesa,309,1769.0


***

### 5. Exportações

Foco:
- Vinhos de mesa
- Espumante

Obs: estamos gerando para todos os tipos caso precise..

#### Vinhos de mesa

In [28]:
df_exportacao_vinho_de_mesa = pd.read_csv('../dados/raw/exportacao/ExpVinho.csv',
                                         delimiter=';')

df_exportacao_vinho_de_mesa = formata_df(df_exportacao_vinho_de_mesa, 'Vinho de mesa')
df_exportacao_vinho_de_mesa.tail()

,ano,pais_origem,pais_destino,tipo,quantidade,valor
6443,2021,Brasil,Tuvalu,Vinho de mesa,0,0
6444,2021,Brasil,Uruguai,Vinho de mesa,136774,149842
6445,2021,Brasil,Vanuatu,Vinho de mesa,0,0
6446,2021,Brasil,Venezuela,Vinho de mesa,26415,35944
6447,2021,Brasil,Vietnã,Vinho de mesa,0,0


#### Espumantes

In [31]:
df_exportacao_espumantes = pd.read_csv('../dados/raw/exportacao/ExpEspumantes.csv',
                                         delimiter=';')

df_exportacao_espumantes = formata_df(df_exportacao_espumantes, 'Espumante')
df_exportacao_espumantes.tail()

,ano,pais_origem,pais_destino,tipo,quantidade,valor
5039,2021,Brasil,Uruguai,Espumante,0,0
5040,2021,Brasil,Vanuatu,Espumante,0,0
5041,2021,Brasil,Venezuela,Espumante,0,0
5042,2021,Brasil,Vietnã,Espumante,144,374
5043,2021,Brasil,Outros(1),Espumante,0,0


#### Uvas frescas

In [33]:
df_exportacao_uvas_frescas = pd.read_csv('../dados/raw/exportacao/ExpUva.csv',
                                         delimiter=';')

df_exportacao_uvas_frescas = formata_df(df_exportacao_uvas_frescas, 'Uvas frescas')
df_exportacao_uvas_frescas.tail()

,ano,pais_origem,pais_destino,tipo,quantidade,valor
6339,2021,Brasil,Uruguai,Uvas frescas,95722,155667
6340,2021,Brasil,Vanuatu,Uvas frescas,73,133
6341,2021,Brasil,Venezuela,Uvas frescas,0,0
6342,2021,Brasil,Vietnã,Uvas frescas,100,271
6343,2021,Brasil,Provisão de Navios e Aeronaves,Uvas frescas,0,0


#### Suco de uva

In [36]:
df_exportacao_suco_de_uva = pd.read_csv('../dados/raw/exportacao/ExpSuco.csv',
                                         delimiter=';')

df_exportacao_suco_de_uva = formata_df(df_exportacao_suco_de_uva, 'Suco de uva')
df_exportacao_suco_de_uva.tail(10)

,ano,pais_origem,pais_destino,tipo,quantidade,valor
5970,2021,Brasil,Tailândia,Suco de uva,2,4
5971,2021,Brasil,Taiwan (Formosa),Suco de uva,60813,122005
5972,2021,Brasil,Tanzânia,Suco de uva,0,0
5973,2021,Brasil,"Tcheca, República",Suco de uva,0,0
5974,2021,Brasil,Togo,Suco de uva,0,0
5975,2021,Brasil,Trinidade e Tobago,Suco de uva,0,0
5976,2021,Brasil,Turquia,Suco de uva,0,0
5977,2021,Brasil,Uruguai,Suco de uva,0,0
5978,2021,Brasil,Vanuatu,Suco de uva,12,8
5979,2021,Brasil,Venezuela,Suco de uva,0,0


#### Escrevendo tabela solicitada no Tech Challenge

##### Concatenando todos os DFs

In [37]:
df_exportacao = pd.concat([df_exportacao_vinho_de_mesa,
                            df_exportacao_espumantes,
                            df_exportacao_uvas_frescas,
                            df_exportacao_suco_de_uva])

df_exportacao.tipo.value_counts()

Vinho de mesa    6448
Uvas frescas     6344
Suco de uva      5980
Espumante        5044
Name: tipo, dtype: int64

##### Escrevendo versão "formatada"

In [38]:
df_exportacao.to_csv('../dados/formatted/df_exportacao_2.csv', encoding='utf-8', sep='|', index=False)

In [41]:
df_exportacao[(df_exportacao['pais_destino'] == 'Brasil') & (df_exportacao['quantidade'] > 0)]

,ano,pais_origem,pais_destino,tipo,quantidade,valor
6344,2021,Brasil,Brasil,Vinho de mesa,31,46
5996,2019,Brasil,Brasil,Uvas frescas,18,62
6240,2021,Brasil,Brasil,Uvas frescas,70,173


In [42]:
df_exportacao[(df_exportacao['pais_destino'] == 'Brasil') & (df_exportacao['valor'] > 0)]

,ano,pais_origem,pais_destino,tipo,quantidade,valor
6344,2021,Brasil,Brasil,Vinho de mesa,31,46
5996,2019,Brasil,Brasil,Uvas frescas,18,62
6240,2021,Brasil,Brasil,Uvas frescas,70,173


***